In [414]:
%matplotlib ipympl
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [415]:
# method to map from image plane to estimated groud plane (GP)
# 1) for each KF, select visible points from map_points
# 2) get four map_points from subset with smallest distance from GP
# 3) project these points onto the GP
# 4) use cv2.getPerspectiveTransform() to compute a homography between the four selected and projected GP points and their correpsonding image points in the KF
# 5) project KF image onto GP by using cv2.warpPerspective()

# alternatives:
# 2b) get N>4 (e.g. N=10) map_points with smallest distance from GP
# 3b) project points onto GP
# 4b) use cv2.findHomography() to estimate the best fit homography via RANSAC

In [416]:
map_points = pickle.load(open("../map_points.pkl", "rb"))
kf_visible_map_points = pickle.load(open("../kf_visible_map_points.pkl", "rb"))
kf_poses = pickle.load(open("../kf_poses.pkl", "rb"))
kf_frames = pickle.load(open("../kf_frames.pkl", "rb"))
kf_kp_matched = pickle.load(open("../kf_kp_matched.pkl", "rb"))

### Fit a Plane into the map points using a RANSAC scheme

In [417]:
import numpy as np
import numpy.linalg as la

eps = 0.00001

def svd(A):
    u, s, vh = la.svd(A)
    S = np.zeros(A.shape)
    S[:s.shape[0], :s.shape[0]] = np.diag(s)
    return u, S, vh


def fit_plane_LSE(points):
    # points: Nx4 homogeneous 3d points
    # return: 1d array of four elements [a, b, c, d] of
    # ax+by+cz+d = 0
    assert points.shape[0] >= 3 # at least 3 points needed
    U, S, Vt = svd(points)
    null_space = Vt[-1, :]
    return null_space


def get_point_dist(points, plane):
    # return: 1d array of size N (number of points)
    dists = np.abs(points @ plane) / np.sqrt(plane[0]**2 + plane[1]**2 + plane[2]**2)
    return dists


def fit_plane_LSE_RANSAC(points, iters=1000, inlier_thresh=0.05, num_support_points=None, return_outlier_list=False):
    # points: Nx4 homogeneous 3d points
    # num_support_points: If None perform LSE fit with all points, else pick `num_support_points` random points for fitting
    # return: 
    #   plane: 1d array of four elements [a, b, c, d] of ax+by+cz+d = 0
    #   inlier_list: 1d array of size N of inlier points
    max_inlier_num = -1
    max_inlier_list = None
    
    N = points.shape[0]
    assert N >= 3

    for i in range(iters):
        chose_id = np.random.choice(N, 3, replace=False)
        chose_points = points[chose_id, :]
        tmp_plane = fit_plane_LSE(chose_points)
        
        dists = get_point_dist(points, tmp_plane)
        tmp_inlier_list = np.where(dists < inlier_thresh)[0]
        tmp_inliers = points[tmp_inlier_list, :]
        num_inliers = tmp_inliers.shape[0]
        if num_inliers > max_inlier_num:
            max_inlier_num = num_inliers
            max_inlier_list = tmp_inlier_list
        
        #print('iter %d, %d inliers' % (i, max_inlier_num))

    final_points = points[max_inlier_list, :]
    if num_support_points:
        max_support_points = np.min((num_support_points, final_points.shape[0]))
        support_idx = np.random.choice(np.arange(0, final_points.shape[0]), max_support_points, replace=False)
        support_points = final_points[support_idx, :]
    else:
        support_points = final_points
    print(final_points.shape)
    plane = fit_plane_LSE(support_points)
    
    fit_variance = np.var(get_point_dist(final_points, plane))
    print('RANSAC fit variance: %f' % fit_variance)
    print(plane)

    dists = get_point_dist(points, plane)

    select_thresh = inlier_thresh * 1

    inlier_list = np.where(dists < select_thresh)[0]
    if not return_outlier_list:
        return plane, inlier_list
    else:
        outlier_list = np.where(dists >= select_thresh)[0]
        return plane, inlier_list, outlier_list

In [418]:
map_points_h = cv2.convertPointsToHomogeneous(map_points).reshape(-1, 4)

In [419]:
import time

In [420]:
t0 = time.perf_counter()
plane, inlier_list = fit_plane_LSE_RANSAC(map_points_h, iters=1000, inlier_thresh=1, num_support_points=3000, return_outlier_list=False)
dt = time.perf_counter() - t0
print(inlier_list.shape, dt)

(30891, 4)
RANSAC fit variance: 0.042779
[-2.57682276e-04  2.87782906e-04 -5.05325584e-02  9.98722339e-01]
(30480,) 1.0236213621683419


### Project map points onto plane along the plane normal

In [421]:
# project all map points onto the plane
def plane_to_hessian(plane):
    """Convert plane to Hessian normal form (n.x + p = 0)"""
    a, b, c, d = plane
    nn = np.sqrt(a**2+b**2+c**2)
    n = np.array([a/nn, b/nn, c/nn])
    p = d/nn
    return n, p

def project_points(plane, points):
    """Project points with shape (-1, 3) onto plane (given as coefficients a, b, c, d with ax+by+cz+d=0)."""
    n, p = plane_to_hessian(plane)
    p_orig = -n*p
    points_proj = points.reshape(-1, 3) - (np.sum(n*(points.reshape(-1, 3) - p_orig.reshape(1, 3)), axis=1)*n.reshape(3, 1)).T
    return points_proj

In [422]:
map_points_proj = project_points(plane, map_points[inlier_list])

In [423]:
map_points_proj

array([[ -8.23663722,  -0.72573774,  19.80180605],
       [-15.15508903,  -3.60271266,  19.82070115],
       [  4.93865111,   7.37936134,  19.78077942],
       ...,
       [ 13.24063593,  -0.78712295,  19.69193672],
       [ 28.5774711 ,   2.65779449,  19.63334791],
       [  4.9987895 ,   0.84780243,  19.74327553]])

### Transform map points from world coordinates to plane coordinates

Choose a random orthonormal base inside the plane. Find an affine transformation M that maps 3D world coordinates of the projected map points (projected onto the plane) to 2D plane coordinates. These plane coordinates are defined with respect to the orthonormal base. Based on this stackoverflow question: https://stackoverflow.com/a/18522281

In [424]:
# transform points from world to plane coordinates
def get_world2plane_transformation(plane, points, return_base=True):
    """Yields an affine transformation M from 3D world to 2D plane coordinates.

    Args:
    
        plane (`numpy.ndarray`): Shape (4,). Plane coefficients [a, b, c, d] which fullfill
            ax + by + cz + d = 0.

        points (`numpy.ndarray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
        
        return_base (`bool`): Wether to return the orthonormal base or not.
    
    Returns:
    
        M (`numpy.ndarray`): Shape (4, 4). Affine transformation matrix which maps points on the plane
        from 3D world coordinates to 2D points with respect to a randomly chosen orthonormal base on the plane.
        Compute point2D = M @ point3D to map a 3D point to 2D coordinates. To retrieve a 3D point from 2D 
        planes coordinates compute point3D = inv(M) @ point2D.
        
        base (`numpy.ndarray`) Shape (3, 4). right-handend orthonormal base in which 2D plane points are 
        expressed. Column vectors of the array correspond to the origin point O of the base, the first and 
        second base vector u, v and the third base vector n which is the plane normal.  
    """
    # pick a random point on the plane as origin and another one to form first base vector
    point_idx = np.arange(0, points.shape[0])
    np.random.seed(0)
    plane_O, plane_U = points[np.random.choice(point_idx, 2, replace=False), :]
    u = (plane_U - plane_O)/np.linalg.norm(plane_U - plane_O)
    n, _ = plane_to_hessian(plane)  # plane normal
    # compute third base vector
    v = np.cross(u, n)/np.linalg.norm(np.cross(u, n))
    # get end points of base vectors
    U = plane_O + u
    V = plane_O + v
    N = plane_O + n
    # form base quadruplet
    D = np.array([[0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1],
                  [1, 1, 1, 1]])
    # form transformation matrix S with M * S = D
    S = np.stack((np.append(plane_O, 1), np.append(U, 1), np.append(V, 1), np.append(N, 1)), axis=1)
    # compute affine transformation M which maps points from world to plane coordinates
    M = np.matmul(D, np.linalg.inv(S))
    if return_base:
        base = np.stack([plane_O.T, u.T, v.T, n.T], axis=1)
        return M, base
    else:
        return M


def map_points_world2plane(points_world, M):
    """Transforms 3D points on a plane to 2D plane coordinates given the transformation matrix M.
    
    Args:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.

        M (`nnumpy.ndarray`): Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.
    """
    points_world_h = cv2.convertPointsToHomogeneous(points_world).reshape(-1, 4)
    points_plane_h = (M @ points_world_h.T).T
    points_plane = cv2.convertPointsFromHomogeneous(points_plane_h).reshape(-1, 3)
    points_plane = points_plane[:, :2]
    return points_plane


def map_points_plane2world(points_plane, M):
    """Transforms 2D plane points into 3D world coordinates given the transformation matrix M.
    
    Args:
    
        points_plane (`numpy.ndarrray`): Shape (-1, 2). 2D points on the plane in (xp, yp, z=0) plane coordinates 
        w.r.t. to a randomly chosen orthonormal base.

        M (`nnumpy.ndarray`) Shape (4, 4). Affine transformation matrix computed with `get_world2plane_transformation`.
    
    Returns:
    
        points_world (`numpy.ndarrray`): Shape (-1, 3). 3D points on the plane in (x, y, z) world coordinates.
    """
    points_plane_tmp = np.zeros((points_plane.shape[0], 3))
    points_plane_tmp[:, :2] = points_plane
    points_plane_h = cv2.convertPointsToHomogeneous(points_plane_tmp).reshape(-1, 4)
    points_world_h = (np.linalg.inv(M) @ points_plane_h.T).T
    points_world = cv2.convertPointsFromHomogeneous(points_world_h).reshape(-1, 3)
    return points_world

In [425]:
# test functions
points_world = map_points_proj
print(points_world)
M, base = get_world2plane_transformation(plane, points_world)
points_plane = map_points_world2plane(points_world, M)
print(points_plane)
#points_world = map_points_plane2world(points_plane, M)  # yields the original points which shows the mapping is correct
#print(points_world)

[[ -8.23663722  -0.72573774  19.80180605]
 [-15.15508903  -3.60271266  19.82070115]
 [  4.93865111   7.37936134  19.78077942]
 ...
 [ 13.24063593  -0.78712295  19.69193672]
 [ 28.5774711    2.65779449  19.63334791]
 [  4.9987895    0.84780243  19.74327553]]
[[ 11.82000098  35.6359641 ]
 [ 16.0705875   41.80644475]
 [  5.15034169  21.67899666]
 ...
 [ -6.25466693  24.03443486]
 [-17.26870421  12.8192494 ]
 [  1.55577407  27.13292813]]


In [426]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], points_world[:1000, 2], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-20,20])
ax.set_ylim([-20,20])
ax.set_zlim([0,40])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_aspect(1.0)

# plot base points (point_O, U, V, N)
ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")

ax.scatter(0, 0, 0, c="black", s=5)
plt.show() 

FigureCanvasNbAgg()

In [427]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.scatter(points_world[:1000, 0], points_world[:1000, 1], s=1, c="red")
ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")
ax.set_xlim([-30,20])
ax.set_ylim([-20,20])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect(1.0)
ax.scatter(base[0, 0], base[1, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], c="blue")
ax.scatter(0, 0, c="black", s=5)

# base [[Ox, ux, vx, nx],
#       [Oy, uy, vy, ny],
#       [Oz, uz, vz, nz]]

ax.grid()
plt.show() 

FigureCanvasNbAgg()

### Find homography between camera plane and ground plane for each keyframe

In [658]:
# do computations for kf 1
kf_index = 0

In [659]:
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111)
ax.imshow(kf_frames[kf_index])
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [660]:
# convert visible map point range to index list
vis_map_pts_arr = np.array(kf_visible_map_points[kf_index][0])
# get index of first element
first_idx = vis_map_pts_arr[0]
# keep only those points that were marked as inliers during plane fitting
vis_map_pts_arr = np.array([v for v in vis_map_pts_arr if v in inlier_list])
# subtract index of first element to get 0-based indices
vis_map_pts_arr = vis_map_pts_arr - first_idx

In [661]:
vis_map_pts_arr.shape

(1002,)

In [662]:
kf_points_image = kf_kp_matched[kf_index][vis_map_pts_arr]

In [663]:
kf_points_image.shape

(1002, 2)

In [664]:
kf_points_image

array([[ 493.,  491.],
       [  87.,  322.],
       [1270.,  969.],
       ...,
       [1868., 1036.],
       [1548.,  394.],
       [ 949.,  214.]])

In [665]:
kf_points_plane = points_plane[first_idx:first_idx+kf_points_image.shape[0], :]

In [666]:
kf_points_plane.shape

(1002, 2)

In [667]:
kf_points_plane

array([[11.82000098, 35.6359641 ],
       [16.0705875 , 41.80644475],
       [ 5.15034169, 21.67899666],
       ...,
       [-2.3851145 , 15.88632915],
       [-3.49956983, 27.52311549],
       [ 2.74924734, 35.36831432]])

In [668]:
# map from image to camera coordinates
w = 1920
h = 1080
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

kf_points_image = cv2.convertPointsToHomogeneous(kf_points_image).reshape(-1, 3)

In [669]:
kf_points_camera = (np.linalg.inv(camera_matrix) @ kf_points_image.T).T
kf_points_camera = cv2.convertPointsFromHomogeneous(kf_points_camera).reshape(-1, 2)

In [670]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
ax.scatter(kf_points_plane[:, 0], kf_points_plane[:, 1], s=1)
ax.scatter(kf_points_camera[:, 0], kf_points_camera[:, 1], s=1)

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [671]:
# find homography using the plane points and image points (kps)

In [672]:
srcPoints = kf_points_camera
dstPoints = kf_points_plane
H, mask = cv2.findHomography(srcPoints, dstPoints, method=cv2.RANSAC)

In [673]:
H

array([[-1.67876255e+01,  1.08897195e+01,  5.28445165e+00],
       [-1.06708279e+01, -1.63605324e+01,  3.05448344e+01],
       [ 6.05234183e-03,  1.04972577e-02,  1.00000000e+00]])

In [674]:
H1 = H

In [675]:
print("Percentage of inliers: {} %".format(np.sum(mask)/srcPoints.shape[0]*100))

Percentage of inliers: 100.0 %


In [676]:
warped_frame = cv2.warpPerspective(kf_frames[kf_index], H, kf_frames[kf_index].shape[::-1], cv2.INTER_CUBIC)

In [677]:
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111)
ax.imshow(warped_frame)
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [614]:
warped_frame_0 = cv2.warpPerspective(kf_frames[0], H0, kf_frames[0].shape[::-1], cv2.INTER_CUBIC)
warped_frame_1 = cv2.warpPerspective(kf_frames[1], H1, kf_frames[1].shape[::-1], cv2.INTER_CUBIC)

In [618]:
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111)
ax.imshow(((warped_frame_0+warped_frame_1)/2).astype(np.uint8))
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()